In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
# import tensorflow as tf
# import tensorflow.keras as keras
# from tensorflow.keras import layers
import torch
import torch.nn as nn
from scipy import stats

In [2]:
sb.set_theme()

# Prepare datasets

In [3]:
df_x = pd.read_csv('../data/dataset_02/data.csv')
df_y = pd.read_csv('../data/dataset_02/answer.csv')

In [4]:
df_x = df_x.drop(labels=['Unnamed: 0'], axis=1)
df_x

In [5]:
df_y = df_y.drop(labels=['Unnamed: 0'], axis=1)
df_y

In [6]:
df = pd.concat([df_x, df_y], axis=1)
df

In [7]:
labels = [str(i) for i in range(672)]

In [8]:
df = df.drop_duplicates(subset=labels, ignore_index=True)
df

In [9]:
df_x = df.drop(labels=['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], axis=1)
df_x

In [10]:
df_y = df.drop(labels=labels, axis=1)
df_y

In [11]:
df_x = np.reshape(df_x.to_numpy(), (-1, 24, 28, 1))
df_x.shape

In [12]:
df_y = df_y.to_numpy()
df_y

In [13]:
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, train_size=0.9)

train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                    train_y,
                                                    test_size=0.33)

In [14]:
print(train_x.shape), print(train_y.shape)
print(valid_x.shape), print(valid_y.shape)
print(test_x.shape), print(test_y.shape)

# TensorFlow

In [28]:
model_tf = keras.Sequential([
    keras.Input(shape=(24, 28, 1)),
    layers.Conv2D(32, (7, 7), activation="relu", data_format='channels_last'),
    layers.MaxPooling2D((2, 2), strides=2),
    layers.Conv2D(64, (3, 3), activation="relu", data_format='channels_last'),
    layers.MaxPooling2D((2, 2), strides=2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='linear')
])

In [29]:
model_tf.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.MeanSquaredError()])

In [30]:
model_tf.load_weights("../logs/weights_model_02/Weights-026--22.49820.hdf5")

# Prepare data for Torch

In [15]:
df_x = np.reshape(df_x, (-1, 1, 24, 28))
df_x.shape

In [16]:
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, train_size=0.9)

train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                    train_y,
                                                    test_size=0.33)

In [17]:
print(train_x.shape), print(train_y.shape)
print(valid_x.shape), print(valid_y.shape)
print(test_x.shape), print(test_y.shape)

In [18]:
train_x_t = torch.tensor(train_x).float()
train_y_t = torch.tensor(train_y).float()
valid_x_t = torch.tensor(valid_x).float()
valid_y_t = torch.tensor(valid_y).float()
test_x_t = torch.tensor(test_x).float()
test_y_t = torch.tensor(test_y).float()

# Torch

In [19]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, 7)
        self.max_pool_1 = nn.MaxPool2d(2, stride=2)
        self.conv_2 = nn.Conv2d(32, 64, 3)
        self.max_pool_2 = nn.MaxPool2d(2, stride=2)
        self.hidden_1 = nn.Linear(768, 128)
        self.hidden_2 = nn.Linear(128, 64)
        self.hidden_3 = nn.Linear(64, 64)
        self.hidden_4 = nn.Linear(64, 64)
        self.output = nn.Linear(64, 6)

    def forward(self, x):
        x = torch.relu(self.conv_1(x))
        x = self.max_pool_1(x)
        x = torch.relu(self.conv_2(x))
        x = self.max_pool_2(x)
        x = torch.reshape(x, (x.shape[0], 768))
        x = torch.relu(self.hidden_1(x))
        x = torch.relu(self.hidden_2(x))
        x = torch.relu(self.hidden_3(x))
        x = torch.relu(self.hidden_4(x))
        output = self.output(x)
        return output

In [20]:
loss_function = nn.MSELoss()

# Analysis

In [44]:
out = model_tf.evaluate(test_x, test_y, batch_size=64)
out = np.array(out)
out[0]

In [21]:
model = Model()
model.load_state_dict(torch.load("../logs/weights_model_03/model_state_adam_02.pt"))

In [22]:
model.eval()

In [23]:
valid_loss = 0.0
batch_size = 64
for i in range(0, valid_x_t.size()[0], batch_size):
        data = valid_x_t[i:i + batch_size]
        target = valid_y_t[i:i + batch_size]

        prediction = model(data)
        loss = loss_function(prediction, target)
        valid_loss += loss.item()
valid_loss / len(valid_y_t)

In [24]:
test_loss = 0.0
batch_size = 64
for i in range(0, test_x_t.size()[0], batch_size):
        data = test_x_t[i:i + batch_size]
        target = test_y_t[i:i + batch_size]

        prediction = model(data)
        loss = loss_function(prediction, target)
        test_loss += loss.item()
test_loss / len(test_y_t)

In [25]:
prediction = model(test_x_t)

In [26]:
prediction = prediction.detach().numpy()

In [27]:
truth = test_y_t.detach().numpy()

In [54]:
delta = []
for pred, tr in zip(prediction, truth):
    abs_pred = np.array([pred[3] - pred[0], pred[4] - pred[1], pred[5] - pred[1]])
    abs_tr = np.array([tr[3] - tr[0], tr[4] - tr[1], tr[5] - tr[1]])
    pred_norm = np.sqrt(np.sum(np.square(abs_pred)))
    tr_norm = np.sqrt(np.sum(np.square(abs_tr)))
    cos = np.sum(abs_pred * abs_tr) / (pred_norm * tr_norm)
    delta.append(cos)

In [55]:
delta = np.arccos(np.array(delta))
delta

In [30]:
# for i in range(len(delta)):
#     if delta[i] >= 0:
#         delta[i] = np.arccos(delta[i])
#     else:
#         delta[i] = -np.arccos(delta[i])

In [31]:
pd.DataFrame(delta).describe()

In [56]:
for i in range(len(delta)):
    if delta[i] > np.pi:
        delta[i] %= np.pi
    elif delta[i] < -np.pi:
        delta[i] %= np.pi
    elif delta[i] > np.pi/2:
        delta[i] -= np.pi/2
    elif delta[i] < -np.pi/2:
        delta[i] += np.pi/2

In [57]:
delta.shape

In [58]:
delta = delta.reshape((1997, 1))
delta

In [35]:
pd.DataFrame(delta).describe()

In [36]:
tmp = []
for tr in truth:
    t = np.array([tr[3] - tr[0], tr[4] - tr[1], tr[5] - tr[1]])
    t = t / np.sqrt(np.sum(np.square(t)))
    tmp.append(np.array([np.arccos(t[2]), np.arctan(t[1]/t[0]) + np.pi/2]))
tmp = np.array(tmp)
tmp

In [37]:
al = np.concatenate((tmp, delta), axis=1)
al

In [38]:
al = pd.DataFrame(al, columns=["theta", "phi", "Относительное смещение, радианы"])

In [39]:
al.describe()

In [40]:
t_1 = al[(al["theta"] >= 0.0) & (al["theta"] < np.pi/3)]["Относительное смещение, радианы"]
t_2 = al[(al["theta"] >= np.pi/3) & (al["theta"] < 2*np.pi/3)]["Относительное смещение, радианы"]
t_3 = al[(al["theta"] >= 2*np.pi/3) & (al["theta"] < np.pi)]["Относительное смещение, радианы"]
p_1 = al[(al["phi"] >= 0.0) & (al["phi"] < np.pi/3)]["Относительное смещение, радианы"]
p_2 = al[(al["phi"] >= np.pi/3) & (al["phi"] < 2*np.pi/3)]["Относительное смещение, радианы"]
p_3 = al[(al["phi"] >= 2*np.pi/3) & (al["phi"] < np.pi)]["Относительное смещение, радианы"]

In [41]:
sb.displot(x=pd.DataFrame(set(t_2) & set(p_1), columns=["Сектор 11"])["Сектор 11"], kind="hist")
plt.ylabel("Плотность распределения")

In [42]:
pd.DataFrame(set(t_1) & set(p_1))

In [64]:
t.hist(figsize=(15, 10) )

In [66]:
t.boxplot(figsize=(15, 10))
plt.ylabel("Уголовое смещение, радианы")

In [63]:
t = pd.concat([pd.DataFrame(set(t_1) & set(p_1), columns=["Сектор 11"]),
                 pd.DataFrame(set(t_1) & set(p_2), columns=["Сектор 12"]),
                 pd.DataFrame(set(t_1) & set(p_3), columns=["Сектор 13"]),
                 pd.DataFrame(set(t_2) & set(p_1), columns=["Сектор 21"]),
                 pd.DataFrame(set(t_2) & set(p_2), columns=["Сектор 22"]),
                 pd.DataFrame(set(t_2) & set(p_3), columns=["Сектор 23"]),
                 pd.DataFrame(set(t_3) & set(p_1), columns=["Сектор 31"]),
                 pd.DataFrame(set(t_3) & set(p_2), columns=["Сектор 32"]),
                 pd.DataFrame(set(t_3) & set(p_3), columns=["Сектор 33"])], axis=1)

In [42]:
pd.DataFrame(set(t_1) & set(p_1)).describe()

In [59]:
delta = pd.DataFrame(delta, columns=["Относительное смещение"])

In [60]:
delta

In [61]:
sb.displot(x=delta["Относительное смещение, радианы"], kind="hist")
plt.ylabel("Плотность распределения")

In [419]:
delta.std()

In [420]:
delta.mean()

In [62]:
sb.boxplot(delta)
plt.ylabel("Угловое смещение, радианы")

In [422]:
from scipy.stats import shapiro
from scipy.stats import normaltest

In [423]:
shapiro(delta)

In [424]:
normaltest(delta)

In [425]:
0.4/np.pi * 360

In [442]:
from joypy import joyplot

joyplot(t, colormap=sb.color_palette("crest", as_cmap=True));
plt.xlabel('Угловое смещение, радианы');
plt.title("Распределение по секторам");

In [455]:
sb.color_palette(palette='colorblind')
t.boxplot(figsize=(15, 10))
plt.ylabel("Угловое смещение, радианы")